In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# load member-level data
df = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/heritage health prize/hhp_final_data.csv")

# perform feature extraction on year 1 data
data = df[df['Year']=='Y1'].copy()

# features = [i for i in data.columns if i not in ('MemberID', 'DaysInHospital', 'Year', 'trainset')]

"""
We will try to find to find the most important values in these categorical variables:
- Primary Condition Group
- Specialty
- Procedure Group
- Place of Service
"""
l = [
    'pcg0', 'pcg1', 'pcg2', 'pcg3', 'pcg4', 'pcg5', 'pcg6', 'pcg7', 'pcg8', 'pcg9', 'pcg10', 'pcg11', 'pcg12', 'pcg13', 'pcg14', 'pcg15', 'pcg16', 'pcg17', 'pcg18', 'pcg19', 'pcg20', 'pcg21', 'pcg22', 'pcg23', 'pcg24', 'pcg25', 'pcg26', 'pcg27', 'pcg28', 'pcg29', 'pcg30', 'pcg31', 'pcg32', 'pcg33', 'pcg34', 'pcg35', 'pcg36', 'pcg37', 'pcg38', 'pcg39', 'pcg40', 'pcg41', 'pcg42', 'pcg43', 'pcg44', 'pcg45',
    'sp0', 'sp1', 'sp2', 'sp3', 'sp4', 'sp5', 'sp6', 'sp7', 'sp8', 'sp9', 'sp10', 'sp11', 'sp12',
    'pg0', 'pg1', 'pg2', 'pg3', 'pg4', 'pg5', 'pg6', 'pg7', 'pg8', 'pg9', 'pg10', 'pg11', 'pg12', 'pg13', 'pg14', 'pg15', 'pg16', 'pg17',
    'ps0', 'ps1', 'ps2', 'ps3', 'ps4', 'ps5', 'ps6', 'ps7', 'ps8'
]

# add a binary feature `in_hospital`, telling if the patient will visit hospital
def is_in_hospital(col):
  if col>0:
    return 1
  else:
    return 0

data.loc[:, 'in_hospital'] = data['DaysInHospital'].apply(is_in_hospital)

In [ ]:
# Use Random Forest Classifier to find the feature importances. In this case,
# we try to see whether the above categorical values are good in predicting if
# the patient will visit hospital next year
from sklearn.ensemble import RandomForestClassifier

y = data['in_hospital']
X = data[l]

clf = RandomForestClassifier(max_depth=5, random_state=0)
clf.fit(X, y)

importances = clf.feature_importances_
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

# save the feature importances into a dictionary
feature_importance_dict = {'rank': [], 'feature_label': [], 'importance': []}
for f in range(X.shape[1]):
  feature_importance_dict['rank'].append(f + 1)
  feature_importance_dict['feature_label'].append(l[indices[f]])
  feature_importance_dict['importance'].append(importances[indices[f]])
  print("%d. feature %d %s (%f)" % (f + 1, indices[f], l[indices[f]], importances[indices[f]]))

Feature ranking:
1. feature 60 pg1 (0.163894)
2. feature 53 sp7 (0.104699)
3. feature 47 sp1 (0.067891)
4. feature 77 ps0 (0.064771)
5. feature 51 sp5 (0.056775)
6. feature 80 ps3 (0.053761)
7. feature 62 pg3 (0.051650)
8. feature 81 ps4 (0.045779)
9. feature 25 pcg25 (0.045494)
10. feature 64 pg5 (0.039881)
11. feature 59 pg0 (0.038324)
12. feature 48 sp2 (0.032552)
13. feature 65 pg6 (0.029022)
14. feature 78 ps1 (0.027038)
15. feature 8 pcg8 (0.014988)
16. feature 9 pcg9 (0.012958)
17. feature 61 pg2 (0.011537)
18. feature 16 pcg16 (0.009234)
19. feature 10 pcg10 (0.007983)
20. feature 85 ps8 (0.007828)
21. feature 24 pcg24 (0.007765)
22. feature 19 pcg19 (0.006678)
23. feature 7 pcg7 (0.006407)
24. feature 63 pg4 (0.006389)
25. feature 0 pcg0 (0.005929)
26. feature 79 ps2 (0.005261)
27. feature 55 sp9 (0.004532)
28. feature 52 sp6 (0.004072)
29. feature 1 pcg1 (0.003938)
30. feature 6 pcg6 (0.003900)
31. feature 54 sp8 (0.003865)
32. feature 46 sp0 (0.003828)
33. feature 18 pcg18 (

In [ ]:
feature_importance_df = pd.DataFrame(feature_importance_dict)

# only keep the first 50 most important features
omitted_l = feature_importance_df[feature_importance_df['rank']>50]['feature_label']

In [ ]:
features_to_drop = ('MemberID', 'DaysInHospital', 'Year', 'trainset', 'in_hospital') + tuple(omitted_l.values)
features = [i for i in data.columns if i not in features_to_drop]

In [ ]:
df.head()

,MemberID,ClaimsTruncated,DaysInHospital,Year,trainset,age_0-9,age_10-19,age_20-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-79,age_80+,age_nan,sex_F,sex_M,sex_nan,no_claims,no_providers,no_vendors,no_pcp,no_placesvc,no_specialty,no_pcg,no_pg,max_paydelay,min_paydelay,avg_paydelay,std_paydelay,max_los,min_los,avg_los,std_los,los_tot_unknown,los_tot_suppressed,los_tot_known,max_charlsonindex,min_charlsonindex,avg_charlsonindex,...,pg0,pg1,pg2,pg3,pg4,pg5,pg6,pg7,pg8,pg9,pg10,pg11,pg12,pg13,pg14,pg15,pg16,pg17,ps0,ps1,ps2,ps3,ps4,ps5,ps6,ps7,ps8,min_drugcount,max_drugcount,avg_drugcount,count_drugcount,min_labcount,max_labcount,avg_labcount,count_labcount,pcp_prob,vendor_prob,provider_prob,drugcount_nan,labcount_nan
0,24027423,0,0,Y1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,5,3,3,1,2,3,3,4,35,27,30.400000,4.219005,0.0,0.0,0.0,-1.0,5,0,0,4,0,2.8,...,1.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,3.000000,1.0,7.0,7.0,7.000000,1.0,0.257737,0.299153,0.239850,0,0
1,98324177,0,0,Y1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,6,3,3,1,2,2,2,4,146,23,70.666667,59.308234,0.0,0.0,0.0,-1.0,6,0,0,0,0,0.0,...,0.0,2.0,1.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.000000,1.0,7.0,7.0,7.000000,1.0,0.265180,0.248000,0.242471,0,0
2,33899367,1,1,Y1,1,0,0,0,0,0,0,0,0,1,0,1,0,0,41,8,8,1,3,6,8,6,162,13,45.000000,31.216983,0.0,0.0,0.0,-1.0,41,0,0,0,0,0.0,...,5.0,22.0,1.0,3.0,0.0,0.0,1.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,6.0,2.300000,10.0,1.0,1.0,1.000000,1.0,0.106716,0.327027,0.209908,0,0
3,5481382,0,1,Y1,1,0,0,1,0,0,0,0,0,0,0,1,0,0,12,6,5,1,3,5,5,4,162,28,68.000000,48.151277,0.0,0.0,0.0,-1.0,12,0,0,0,0,0.0,...,0.0,5.0,0.0,1.0,0.0,0.0,4.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,3.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,2.0,1.111111,9.0,1.0,2.0,1.666667,3.0,0.213570,0.245260,0.239850,0,0
4,69908334,0,0,Y1,1,0,0,0,0,0,0,1,0,0,0,1,0,0,17,9,6,1,3,5,5,6,77,22,48.117647,16.773798,0.0,0.0,0.0,-1.0,17,0,0,0,0,0.0,...,4.0,6.0,1.0,3.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,4.0,1.777778,9.0,8.0,8.0,8.000000,1.0,0.268316,0.245260,0.253925,0,0


In [ ]:
final_df = df.drop(columns=omitted_l)

final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147473 entries, 0 to 147472
Columns: 110 entries, MemberID to labcount_nan
dtypes: float64(74), int64(35), object(1)
memory usage: 123.8+ MB


In [ ]:
# 36 features have been removed
# final_df.to_csv(r'/content/drive/MyDrive/Colab Notebooks/heritage health prize/final_data.csv', index=False)

In [ ]:
# final_df[['MemberID', 'pcp_prob', 'vendor_prob', 'provider_prob']]

,MemberID,pcp_prob,vendor_prob,provider_prob
0,24027423,0.257737,0.299153,0.239850
1,98324177,0.265180,0.248000,0.242471
2,33899367,0.106716,0.327027,0.209908
3,5481382,0.213570,0.245260,0.239850
4,69908334,0.268316,0.245260,0.253925
...,...,...,...,...
147468,206785,0.294773,0.265155,0.273366
147469,93465383,0.211600,0.327749,0.329606
147470,57466586,0.325972,0.289252,0.272189
147471,57283229,0.000000,0.207957,0.207769
